<a href="https://colab.research.google.com/github/LeandroMartins0/explode-unnest/blob/main/Explode_unnest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configuração de ambiente

In [1]:
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
# configurar as variáveis de ambiente

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [3]:
# tornar o pyspark "importável"

import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

In [4]:
# Importando pyspark e dependencias

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, ArrayType

In [5]:
# Iniciando sessão

spark = SparkSession.builder.appName('SparkNestedFields').getOrCreate()

# Carregando arquivo

In [ ]:
# Carregando JSON

teste_df = sc.read.option("multiline","true").json("/content/test.json")

# Função Explode Unnest

In [ ]:
# Schema

teste_df.printSchema()

In [ ]:
# DataFrame

teste_df.show(truncate=False)

In [ ]:
# 1
# Caso o JSON seja um array ou um map basta utilizar este código

v5_struct_exploded = teste_df.withColumn("v5_struct_exploded", F.explode("v5_struct"))
v5_struct_exploded.show(truncate=False)

In [ ]:
# 2
# Aqui iremos explodir novas colunas e um novo DataFrame

v5_struct_exploded = v5_struct_exploded.select(
     "v1_string",
     "v5_struct_exploded.t5",
     "v5_struct_exploded.t6",)

v5_struct_exploded.show(truncate=False)

In [ ]:
# Aqui temos o mesmo código porém com mais sucinto

teste_df.withColumn("v5_struct_exploded", F.explode(array("v5_struct")))\
            .select("v1_string",
                    "v5_struct_exploded.t5",
                    "v5_struct_exploded.t6",)\
            .show(truncate=False)

In [ ]:
# Exemplo para Struct dentro de Struct

teste_df.withColumn("v7_structs_oneDriveBusiness_exploded", F.explode(array("v7_structs.oneDriveBusiness")))\
            .select("v1_string",
                    "v7_structs_oneDriveBusiness_exploded.account",
                    "v7_structs_oneDriveBusiness_exploded.enabled",
                    "v7_structs_oneDriveBusiness_exploded.folder",
                    "v7_structs_oneDriveBusiness_exploded.folderName",)\
            .show(truncate=False)

# Converter struct em map

In [ ]:
# Transformando de struct para map

from pyspark.sql.functions import col,lit,create_map
df = teste_df.withColumn("v5_structMap",create_map(
        lit("t5"),col("v5_struct.t5"),
        lit("t6"),col("v5_struct.t6")
        )).drop("v5_struct")
df.printSchema()
df.show(truncate=False)

In [ ]:
# Explodindo tabela

v5_structMap_exploded = df.select(
     "v1_string",
     "v5_structMap.t5",)

v5_structMap_exploded.show(truncate=False)